In [67]:
import xarray as xr
import pandas as pd
from pathlib import Path

data_dir = "./GFED5/monthly/" 
start_date = "2018-04"
end_date = "2020-12" 
aggregation = "mean"

# Convert dates to datetime for comparison
start_date = pd.to_datetime(start_date, format="%Y-%m")
end_date = pd.to_datetime(end_date, format="%Y-%m")

# List all NetCDF files in the directory

all_files = sorted(Path(data_dir).glob("*.nc"))

# Filter files by date range

filtered_files = [
     str(file) for file in all_files
     if start_date.year <= int(file.stem[-4:]) <= end_date.year
     ]

if not filtered_files:
     raise ValueError("No files match the specified date range.")

print(f"Found {len(filtered_files)} files in the date range.")

Found 3 files in the date range.


In [68]:
import xarray as xr
import pandas as pd
from pathlib import Path

data_dir = "./GFED5/monthly/" 
start_date = pd.to_datetime("2018-04", format="%Y-%m")
end_date = pd.to_datetime("2020-12", format="%Y-%m")
aggregation = "mean"

year_start = start_date.year
year_end = end_date.year


all_files = sorted(Path(data_dir).glob("*.nc"))

# Filter files by date range

filtered_files = [
     str(file) for file in all_files
     if year_start <= int(file.stem[-4:]) <= year_end
     ]

filtered_files

['GFED5/monthly/GFED5_Beta_monthly_2018.nc',
 'GFED5/monthly/GFED5_Beta_monthly_2019.nc',
 'GFED5/monthly/GFED5_Beta_monthly_2020.nc']

In [69]:
# Open the dataset and select only one data variable
var = "C"
ds = xr.open_mfdataset(filtered_files)[var]


# Aggregate the data by the specified method
ds2 = ds.mean(dim="time")
# ds2 = ds2.sel(
#     lat=slice(37, -35),
#     lon=slice(-17, 51)
# )

ds2

<xarray.DataArray 'C' (lat: 720, lon: 1440)> Size: 4MB
dask.array<mean_agg-aggregate, shape=(720, 1440), dtype=float32, chunksize=(720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 3kB 89.88 89.62 89.38 89.12 ... -89.38 -89.62 -89.88
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9

In [ ]:
import h3
import pandas as pd
import dask.array as da
from collections import defaultdict

# Extract latitude, longitude, and data values as Dask arrays
lat = da.from_array(ds2['lat'].values)  # Shape: (720,)
lon = da.from_array(ds2['lon'].values)  # Shape: (1440,)
data = da.from_array(ds2.values)       # Shape: (720, 1440)

# Define H3 resolution
resolution = 3

# Expand lat and lon to match the shape of data
lat_grid, lon_grid = da.meshgrid(lat, lon, indexing="ij")

# Flatten all arrays
lat_flat = lat_grid.flatten()
lon_flat = lon_grid.flatten()
data_flat = data.flatten()

# Combine lat, lon, and data into a single Dask array
points = da.stack([lat_flat, lon_flat, data_flat], axis=1)

# Define a function to process a chunk of data
def process_chunk(chunk):
    hex_dict = defaultdict(float)
    for lat, lon, value in chunk:
        if value > 0:  # Only process positive values
            hex_id = h3.latlng_to_cell(lat, lon, resolution)
            hex_dict[hex_id] += value  # Sum values directly
    return hex_dict

# Use Dask's map_blocks to process the array in parallel
hex_dicts = points.map_blocks(
    lambda block: [process_chunk(block.compute())],  # Process chunks
    dtype=object
).compute()

# Merge dictionaries into a single dictionary
final_hex_dict = defaultdict(float)
for hex_dict in hex_dicts:
    for hex_id, value in hex_dict.items():
        final_hex_dict[hex_id] += value

# Round values after aggregation
aggregated_hex_data = {hex_id: round(value, 2) for hex_id, value in final_hex_dict.items()}

# Convert to a DataFrame
df = pd.DataFrame(aggregated_hex_data.items(), columns=["hex_id", "value"])

print(df)

AttributeError: 'numpy.ndarray' object has no attribute 'compute'

In [70]:
import h3
import pandas as pd

# Extract latitude, longitude, and data values
lat = ds2['lat'].values  # NumPy array
lon = ds2['lon'].values  # NumPy array
data = ds2.values        # NumPy array

# Define H3 resolution
resolution = 3

# Initialize dictionary for hexagon data
hex_dict = {}

# Process data points, filter positive values, and bin into hexagons
for lat_idx in range(len(lat)):
    for lon_idx in range(len(lon)):
        value = data[lat_idx, lon_idx]
        if value > 0:  # Only process positive values
            hex_id = h3.latlng_to_cell(lat[lat_idx], lon[lon_idx], resolution)
            if hex_id not in hex_dict:
                hex_dict[hex_id] = 0
            hex_dict[hex_id] += value  # Aggregate values directly

# Round values after aggregation
aggregated_hex_data = {hex_id: round(total_value, 2) for hex_id, total_value in hex_dict.items()}


df = pd.DataFrame(aggregated_hex_data.items(), columns=["hex_id", "value"])

df

,hex_id,value
0,830636fffffffff,0.19
1,8305aefffffffff,7.94
2,8305aafffffffff,0.68
3,830b6bfffffffff,0.08
4,8305a3fffffffff,2.26
...,...,...
9722,83cf16fffffffff,0.06
9723,83de70fffffffff,0.03
9724,83df69fffffffff,0.01
9725,83df6bfffffffff,0.22


In [72]:
# plot using pydeck h3_hexagon_layer

import pydeck as pdk

# Define the layer

layer = pdk.Layer(
     "H3HexagonLayer",
     data=df,
     pickable=True,
     stroked=True,
     filled=True,
     extruded=True,
     get_hexagon="hex_id",
     get_fill_color="[255 - value, 255, value]",
     get_line_color=[0, 0, 0],
     line_width_min_pixels=2,
     )


# Set the viewport location
view_state = pdk.ViewState(
     latitude=0,
     longitude=0,
     zoom=2,
     bearing=0,
     pitch=0,
     )

# Render the map
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r


{
  "initialViewState": {
    "bearing": 0,
    "latitude": 0,
    "longitude": 0,
    "pitch": 0,
    "zoom": 2
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "hex_id": "830636fffffffff",
          "value": 0.1899999976158142
        },
        {
          "hex_id": "8305aefffffffff",
          "value": 7.940000057220459
        },
        {
          "hex_id": "8305aafffffffff",
          "value": 0.6800000071525574
        },
        {
          "hex_id": "830b6bfffffffff",
          "value": 0.07999999821186066
        },
        {
          "hex_id": "8305a3fffffffff",
          "value": 2.259999990463257
        },
        {
          "hex_id": "830585fffffffff",
          "value": 2.859999895095825
        },
        {
          "hex_id": "830410fffffffff",
          "value": 0.5400000214576721
        },
        {
          "hex_id": "8305a2fffffffff",
          "value": 46.709999084472656
        },
        {
          "hex_id": "830413fffffffff",
          "value": 102.66000366210938
        },
        {
          "hex_id": "83041efffffffff",
          "value": 383.44000244140625
        },
        {
          "hex_id": "830b6afffffffff",
          "value": 13.550000190734863
        },
        {
          "hex_id": "8305a6fffffffff",
          "value": 26.399999618530273
        },
        {
          "hex_id": "830b4cfffffffff",
          "value": 5.96999979019165
        },
        {
          "hex_id": "830412fffffffff",
          "value": 38.619998931884766
        },
        {
          "hex_id": "8304adfffffffff",
          "value": 40.439998626708984
        },
        {
          "hex_id": "830b41fffffffff",
          "value": 16.670000076293945
        },
        {
          "hex_id": "8305b5fffffffff",
          "value": 216.97999572753906
        },
        {
          "hex_id": "83058efffffffff",
          "value": 142.75999450683594
        },
        {
          "hex_id": "83058bfffffffff",
          "value": 28.540000915527344
        },
        {
          "hex_id": "83041afffffffff",
          "value": 33.040000915527344
        },
        {
          "hex_id": "830b6efffffffff",
          "value": 14.84000015258789
        },
        {
          "hex_id": "83058afffffffff",
          "value": 0.9100000262260437
        },
        {
          "hex_id": "830d0dfffffffff",
          "value": 0.6200000047683716
        },
        {
          "hex_id": "830a5bfffffffff",
          "value": 0.5199999809265137
        },
        {
          "hex_id": "83059dfffffffff",
          "value": 113.43000030517578
        },
        {
          "hex_id": "8304a4fffffffff",
          "value": 539.239990234375
        },
        {
          "hex_id": "8304a5fffffffff",
          "value": 18.639999389648438
        },
        {
          "hex_id": "8304acfffffffff",
          "value": 60.34000015258789
        },
        {
          "hex_id": "8304f4fffffffff",
          "value": 9.069999694824219
        },
        {
          "hex_id": "8304f5fffffffff",
          "value": 6.239999771118164
        },
        {
          "hex_id": "830a5afffffffff",
          "value": 24.350000381469727
        },
        {
          "hex_id": "830583fffffffff",
          "value": 9.739999771118164
        },
        {
          "hex_id": "8304a1fffffffff",
          "value": 405.19000244140625
        },
        {
          "hex_id": "8304a9fffffffff",
          "value": 6.0
        },
        {
          "hex_id": "830124fffffffff",
          "value": 1.2100000381469727
        },
        {
          "hex_id": "830599fffffffff",
          "value": 77.25
        },
        {
          "hex_id": "8304a0fffffffff",
          "value": 1014.9500122070312
        },
        {
          "hex_id": "8304a8fffffffff",
          "value": 35.95000076293945
        },
        {
          "hex_id": "830d70fffffffff",
          "value": 0.019999999552965164
        },
        {
          "hex_id"

In [17]:
import rioxarray as rio
import numpy as np
import leafmap


# ds2 = ds2.where(ds2 != 0, np.nan)


# construct the hexagon binning from ds2 using h3 library




ds2 = ds2.rio.write_crs("EPSG:4326")
ds2 = ds2.rio.set_spatial_dims(x_dim="lon", y_dim="lat")

ds2.rio.to_raster("data.tif", dtype="float32")
# do hexagon binning from the raster data




# m = leafmap.Map(zoom=2)
# m.add_raster(
#     "data.tif", 
#     layer_name="Aggregated Data", 
#     colormap="ocean_r",
#     opacity=0.9
# )

# m

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import xarray as xr

geometries = []
values = []
for hex_id, agg_value in aggregated_hex_data.items():
    boundary = h3.cell_to_boundary(hex_id)
    polygon = Polygon([(y, x) for x, y in boundary])  # Mirror coordinates
    geometries.append(polygon)
    values.append(agg_value)

# Step 6: Create a GeoDataFrame
gdf = gpd.GeoDataFrame(
    {'hex_id': list(aggregated_hex_data.keys()), var: values, 'geometry': geometries},
    geometry='geometry'
)

# Step 7: Export to GeoJSON
gdf.to_file("output.geojson", driver="GeoJSON")

In [ ]:
import folium

m = folium.Map(location=[0, 0], zoom_start=2)

folium.Choropleth(
     geo_data="output.geojson",
     name="choropleth",
     data=gdf,
     columns=["hex_id", var],
     key_on="feature.properties.hex_id",
     fill_color="YlGn",
     fill_opacity=0.7,
     line_opacity=0.2,
     legend_name=var,
     ).add_to(m)

m

In [ ]:
from dask.dataframe import from_pandas

import time
start = time.time()
data = from_pandas(ds2.to_dataframe().reset_index(), npartitions=1000).compute(schedule="threads")
end = time.time()
print("Time taken: ", end-start)

In [ ]:
data

In [ ]:
# plot with leafmap

In [ ]:
data = data.groupby("hex_id", as_index=False).agg({"C": "sum"})

data['bound'] = data['hex_id'].apply(lambda x: h3.cell_to_boundary(x))

# mirror and convert to geojson format
data['bound'] = data['bound'].apply(lambda x: [(y, x) for x, y in x])

import geopandas as gpd
from shapely.geometry import Polygon

data['geometry'] = data['bound'].apply(lambda x: Polygon(x))


gdf = gpd.GeoDataFrame(
    data[['hex_id', 'geometry', 'C']],
    geometry='geometry'
)

gdf.to_file("gfed.geojson", driver='GeoJSON')

In [ ]:
# plot with pydeck h3hexagon layer

import pydeck as pdk

layer = pdk.Layer(
     "H3HexagonLayer",
     data=gdf,
     get_hexagon="geometry",
     get_fill_color="[255, 255, C*10]",
     auto_highlight=True,
     elevation_scale=1000,
     pickable=True,
     extruded=True,
     coverage=1,
     )
